In [1]:
import TAB_generator
from TAB_generator import *
import os
path = os.getcwd()

In [2]:
p, f, t = readTAB(path + '\\test\\')

#####0. a[Air]鸟之诗1^9I.xml
#####1. a[圣斗士]原谅我 1^14A.xml
#####2. A[日在校园]悲しみの向こうへ(向着悲伤的彼方)18^26B.xml
#####3. a[植物大战僵尸]Grasswalk1^8A.xml
#####4. a[植物大战僵尸]Graze the Roof11^18A.xml
#####5. a[植物大战僵尸]植物保龄球 1^13A.xml
#####6. a[猎人]魂魄之挽歌C调 10^17A.xml
    ! In measure 0, sum of timesig should be 3.0, but6.0
    ! In measure 1, sum of timesig should be 3.0, but6.0
    ! In measure 2, sum of timesig should be 3.0, but6.0
    ! In measure 3, sum of timesig should be 3.0, but6.0
    ! In measure 4, sum of timesig should be 3.0, but6.0
    ! In measure 5, sum of timesig should be 3.0, but6.0
    ! In measure 6, sum of timesig should be 3.0, but6.0
    ! In measure 7, sum of timesig should be 3.0, but6.0
#####7. A[秒速五厘米]One more time One more chance1^4I.xml
#####8. a[钢炼]兄弟3^10A.xml
#####9. b[CLANNAD]欢乐岛1^15I.xml
#####10. b[圣斗士]真红的少年 1^16A.xml
#####11. b[魔法少女小圆]Connect18^33A.xml
#####12. c#[无头骑士异闻录]コンプリケイション1^8I.xml
#####13. C[Air]てんとう虫(瓢虫)2^5I.xml
#####14. C[CLANNAD]东风1^9I.xml
#####15. C[Little busters]S

In [3]:
print(p[2][1])
print(f[2][1])

['4B 4D 3E', '4A', '4G# 4D 3E', '4A 3E', '4E 2A', '4E 2A', '4E']
['(1,7) (3,7) (5,7)', '(2,10)', '(2,9) (3,7) (5,7)', '(2,10) (5,7)', '(1,0) (6,5)', '(1,0) (5,0)', '(1,0)']


In [4]:
all_chord = []
for songnum, song in enumerate(p):
    for measurenum, measure in enumerate(song):
        for pitchnum, pitchcu in enumerate(measure):
            if len(pitchcu.split()) >= 3 and int(f[songnum][measurenum][pitchnum].split()[0][1])>1:
                all_chord.append((pitchcu, f[songnum][measurenum][pitchnum]))
print(len(all_chord))               

228


In [5]:
import random
all_chord = random.sample(all_chord, 128)

In [6]:
all_chord

[('4A 4D 2B', '(2,10) (3,7) (6,7)'),
 ('4C 3G 3E', '(2,1) (3,0) (4,2)'),
 ('4C 3G 2G', '(2,1) (3,0) (6,3)'),
 ('3B 3G 2E', '(2,2) (3,0) (6,2)'),
 ('4C 3G 2G', '(2,1) (3,0) (6,3)'),
 ('4C 3E 2F', '(2,3) (4,2) (6,1)'),
 ('4C 3G 3C', '(2,1) (3,0) (5,3)'),
 ('4A 4D 3A', '(2,10) (3,7) (4,7)'),
 ('4F# 3B 2G#', '(2,7) (3,4) (6,4)'),
 ('3A 3D 2B', '(3,2) (4,0) (5,2)'),
 ('4C# 3A 3E 2A', '(2,2) (3,2) (4,2) (5,0)'),
 ('4D 3A 2F', '(2,3) (3,2) (6,1)'),
 ('3B 3F# 2B', '(3,4) (4,4) (5,2)'),
 ('5B 5F# 5D 4G', '(2,12) (3,11) (4,12) (5,10)'),
 ('4F 4D 3A#', '(2,6) (3,7) (4,8)'),
 ('4C 3G 3E 3C', '(2,1) (3,0) (4,2) (5,3)'),
 ('3A 3F# 2B', '(3,2) (4,4) (5,2)'),
 ('4A 4E 3F#', '(2,10) (3,9) (5,9)'),
 ('4D 3A 2B', '(3,7) (4,7) (6,7)'),
 ('4D# 4C 2G#', '(2,4) (3,5) (6,4)'),
 ('4D 3A 2F', '(2,3) (3,2) (6,1)'),
 ('4F 4D 2A#', '(2,6) (3,7) (6,6)'),
 ('4D 3G# 3E 2B 2E', '(2,3) (3,1) (4,2) (5,2) (6,0)'),
 ('4C 3G 3C', '(2,1) (3,0) (5,3)'),
 ('4C 3G 3E 3C', '(2,1) (3,0) (4,2) (5,3)'),
 ('3B 3F# 3C#', '(2,0) (4,4

In [7]:
pitchset, fingerset, timesigset = [], [], []

count = 0
pitchmeasure = []
fingermeasure = []
for chord in all_chord:
    count += 1
    pitchmeasure.append(chord[0])
    fingermeasure.append(chord[1])
    
    if count == 8:
        count=0
        pitchset.append(pitchmeasure)
        fingerset.append(fingermeasure)
        pitchmeasure = []
        fingermeasure = []
        
for i in range(16):
    timesigset.append([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])
generateTAB(pitchset,fingerset,  timesigset, filepath=path + '\\head.xml')

文件名为：head.xml
文件已保存为C:\Users\28056\Desktop\研究生\AnimeTAB\AnimeTAB\new_head.xml


In [8]:
generateTAB(p[2], f[2], t[2], filepath=path + '\\head.xml')

In [15]:
print(p[2])
print(f[2])

[['4A 4E 3F#', '4G#', '4A 4E 3F#', '5C# 3F#', '4E 3D', '4E 3D', '5C#'], ['4B 4D 3E', '4A', '4G# 4D 3E', '4A 3E', '4E 2A', '4E 2A', '4E'], ['4D 3D', '4E', '4F#', '4G#', '4A 3C#', '4E'], ['4F# 3B 2G#', '4B', '4A', '4G# 4D# 3C#', '4F#', '4G#'], ['4A 4E 3F#', '4G#', '4A 4E 3F#', '5C# 3F#', '4E 3D', '4E 3D', '5C#'], ['4B 4D 3E', '4A', '4G# 4D 3E', '4A 3E', '4E 2A', '4E 2A', '4E'], ['4D 2B', '4E', '4F#', '4G#', '4A 3C#', '4G#'], ['4F# 3D', '5C#', '4B', '4A', '4G#', '4A'], ['4G#', '4F# 4C# 3A# 3F# 3C#']]
[['(2,10) (3,9) (5,9)', '(2,9)', '(2,10) (3,9) (5,9)', '(1,9) (5,9)', '(3,9) (6,10)', '(3,9) (6,10)', '(1,9)'], ['(1,7) (3,7) (5,7)', '(2,10)', '(2,9) (3,7) (5,7)', '(2,10) (5,7)', '(1,0) (6,5)', '(1,0) (5,0)', '(1,0)'], ['(2,3) (4,0)', '(1,0)', '(1,2)', '(1,4)', '(1,5) (5,4)', '(1,0)'], ['(2,7) (3,4) (6,4)', '(1,7)', '(1,5)', '(1,4) (2,4) (5,4)', '(2,7)', '(1,4)'], ['(2,10) (3,9) (5,9)', '(2,9)', '(2,10) (3,9) (5,9)', '(1,9) (5,9)', '(3,9) (6,10)', '(3,9) (6,10)', '(1,9)'], ['(1,7) (3,7) (5,

In [9]:
print(pitchset,fingerset,  timesigset)

[['4A 4D 2B', '4C 3G 3E', '4C 3G 2G', '3B 3G 2E', '4C 3G 2G', '4C 3E 2F', '4C 3G 3C', '4A 4D 3A'], ['4F# 3B 2G#', '3A 3D 2B', '4C# 3A 3E 2A', '4D 3A 2F', '3B 3F# 2B', '5B 5F# 5D 4G', '4F 4D 3A#', '4C 3G 3E 3C'], ['3A 3F# 2B', '4A 4E 3F#', '4D 3A 2B', '4D# 4C 2G#', '4D 3A 2F', '4F 4D 2A#', '4D 3G# 3E 2B 2E', '4C 3G 3C'], ['4C 3G 3E 3C', '3B 3F# 3C#', '4D 3G 2G', '4C 3G 3E', '4G# 4D 3E', '4F 4D 3A#', '5C 4A 3A', '4C 3A 3F'], ['5D 4A 4F 3B', '4D 3G 2G', '4B 4G 4E', '4C 3G 3E', '4C 3G 3F#', '4C 3A 3F', '4C# 3F# 3D#', '4D 3G 3E'], ['4C 3G 2E', '3B 3F# 3C#', '4E 3A 3F# 2B', '4C 3G 2G', '3B 3G 3D', '4C 3A 3F', '4A 4E 3F#', '3G 3D 2A 2F'], ['4D 3A 3D 2F#', '5C 4A 3A', '4D 3A 3F 2F', '3G 3E 3C', '4C 3F 2F', '4C 3A 2F', '5C 4G 4E', '4D# 4C 2G#'], ['3B 3G 2G', '4C 3G 3E 3C', '4B 4G 3E', '3A 3G 2A', '4C 3G 3E 3C', '4D# 4C 3G#', '4D 3A 3E', '4B 4G 4E'], ['4D 3A 3G 3D', '4D 3G 3D 2G', '4E 3A 3E 2F#', '4B 4G 3G', '5B 5F# 5D 4G', '3G 3D 2E', '3B 3A 2A', '4C 3G 3C'], ['4C 3G 3E', '4C 3G 2G', '3A# 3F# 3

In [10]:
all_chord[0]

('4A 4D 2B', '(2,10) (3,7) (6,7)')

In [11]:
midi2str(finger2midi('(2,2)'))

'4C#'

In [12]:
from xml.dom import minidom
from xml.dom.minidom import parse
domtree = parse(path + '\\new_head.xml')
notelist = make_notes(domtree, pitch_cu='4C 3C', finger_cu='(2,1) (5,3)', timesig=0.5, divisions=4)

In [13]:
first_measure = domtree.getElementsByTagName('measure')[0]
sum([int(node.getElementsByTagName('duration')[0].firstChild.data) for node in first_measure.childNodes if node.nodeName=='note'])

ValueError: invalid literal for int() with base 10: '2.0'

In [ ]:
notelist[0].getElementsByTagName('duration')[0].firstChild.data

In [ ]:
typedict = {
            'whole':1,
            'half':0.5,
            'quarter':0.25,
            'eighth':0.125,  
            '16th':0.0625
        }
typename = lambda type1: [key for key, value in typedict.items() if value == type1]

In [ ]:
typedict.values()

In [ ]:
float(0.5)